Cerebras GPT

In [1]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("cerebras/Cerebras-GPT-111M")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = AutoModelForCausalLM.from_pretrained("cerebras/Cerebras-GPT-111M")

In [2]:
# mps device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
model = model.to(device)
model.device

device(type='cuda', index=0)

In [4]:
# prompt = 'How many enterprise Tactics IDs are in the Mitre Att&ck v13 matrix released in April 2023?\n\n Assistant:'
# prompt = """ 
# Explain the following payload. Payload is GET /jenkins/postnuke/index.php?module=My_eGallery&do=showpic&pid=-1/**/AND/**/1=2/**/UNION/**/ALL/**/SELECT/**/0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,concat(0x3C7230783E,pn_uname,0x3a,pn_pass,0x3C7230783E),0,0,0/**/FROM/**/md_users/**/WHERE/**/pn_uid=$id/* HTTP/1.1 Content-Length: 36 Content-Type: application/x-www-form-urlencoded Host: www.test.go.kr Connection: Keep-Alive User-Agent: Mozilla/5.00 (Nikto/2.1.6) (Evasions:None).\n\nAssistant:
# """
prompt = 'User: What is xss attack?\n\nAssistant: '

In [5]:
from transformers import pipeline

# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device = 0)

generated_text = pipe(prompt, max_length=256, do_sample=False, no_repeat_ngram_size=2)[0]
print(generated_text['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


User: What is xss attack?

Assistant: 
   - I want to know what is the problem.
- I have a question about the xs attack.  I don't know how to solve it. I know I can't find a solution. But I'm not sure how I could solve this problem, I just want it to work.




Pytorch based inference

In [6]:
# Encode the prompt using the tokenizer
input_ids = tokenizer.encode(prompt, return_tensors="pt")

In [7]:
# Generate text based on the encoded prompt
with torch.no_grad():
    output = model.generate(
        # input_ids=input_ids,
        input_ids=input_ids.to(device),

        do_sample=True,
        top_p=0.75,
        top_k=85,
        temperature=1.99,
        typical_p=1,
        repetition_penalty=1.3,
        max_length=250,  # The maximum number of tokens to generate
        num_beams=5,    # The number of beams to use for beam search
#         early_stopping=True,  # Stop generation when the model predicts an end-of-sequence token
    )

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [8]:
# Decode the generated text and print it
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

User: What is xss attack?

Assistant: 

Command Line: 

Line 2: Command Line

Line 3: Commands Line 1--1

Command Line 2: Command Line

Command Line 3: Command Line

Command Line 4: Command Line

Command Line 8: Command Line

Command Line 8: Command Line

Command Line 9: Control Line

Command Line 10: Control Line

Command Line 11: Control Line

Command Line 14: Command Line

Command Line 12: Control Line

Command Line 15: Control Line

Command Line 16: Command Line

Command Line 16: Command Line

Command Line 17: Command Line

Command Line 18: Command Line

Command Line 20: Command Line

Command Line 22: Command Line

Command Line 24: Command Line

Command Line 25: Command Line

Command Line 26: Command Line

Command Line 27: Command Line

Command Line 28: Command Line

Command Line 30: Command Line

Command Line 31: Command Line

Command Line 34: Command Line

Command Line 36: Command Line

Command Line 37:


Pytorch based fine tuning

In [9]:
import datasets
dataset = datasets.load_dataset('json', data_files='security_base_sample.json', field='train')

Found cached dataset json (/home/ngcsm/.cache/huggingface/datasets/json/default-674a5955815d9130/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'input'],
        num_rows: 43
    })
})

In [11]:
cutoff_len = 1024

def generate_prompt(entry):
    if entry['input']:
        return f"User: {entry['instruction']}: {entry['input']}\n\nAssistant: {entry['output']}"
    else:
        return f"User: {entry['instruction']}\n\nAssistant: {entry['output']}"

def tokenize(item, add_eos_token=True):
    result = tokenizer(
        generate_prompt(item),
        truncation=True,
        max_length=cutoff_len,
        padding=False,
        return_tensors=None,
    )

    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < cutoff_len
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    result["labels"] = result["input_ids"].copy()

    return result

In [12]:
train_val = dataset["train"].train_test_split(test_size=0.05, shuffle=True, seed=595)
train_data = train_val["train"].shuffle().map(tokenize)
val_data = train_val["test"].shuffle().map(tokenize)

Loading cached split indices for dataset at /home/ngcsm/.cache/huggingface/datasets/json/default-674a5955815d9130/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-6423add41a120364.arrow and /home/ngcsm/.cache/huggingface/datasets/json/default-674a5955815d9130/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-5d398ee80083a909.arrow


Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [13]:

if 'model' in globals(): 
    del model
    # torch.cuda.empty_cache()

model = transformers.AutoModelForCausalLM.from_pretrained(
    'cerebras/Cerebras-GPT-111M',    
    
    load_in_8bit=True,
    torch_dtype=torch.float16,

    device_map={'': 0}
    # device = torch.device("cpu")
    # device_map = 'auto'
)



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/ngcsm/.local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/home/ngcsm/.local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)


In [14]:
import peft

model = peft.prepare_model_for_int8_training(model)

model = peft.get_peft_model(model, peft.LoraConfig(
    r=8,
    lora_alpha=16,
    # target_modules=["q_proj", "v_proj"],
    target_modules=["c_attn"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
))


In [15]:
import os
import wandb 

output_dir = 'cerebras-gpt111m-finetune'

use_wandb = True,
wandb_run_name = f"{output_dir}-{wandb.util.generate_id()}"

# set the wandb project where this run will be logged
os.environ["WANDB_PROJECT"]=output_dir

# save your trained model checkpoint to wandb
os.environ["WANDB_LOG_MODEL"]="true"

# turn off watch to log faster
os.environ["WANDB_WATCH"]="false"

In [16]:
training_args = transformers.TrainingArguments(
    # per_device_train_batch_size=16, 
    per_device_train_batch_size=1, 

    gradient_accumulation_steps=8,  
    num_train_epochs=30,  
    learning_rate=1e-4, 
    # only be used on CUDA devices.
    fp16=True,
    
    optim="adamw_torch",
    logging_steps=10, 
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=200,
    save_steps=200,
    output_dir=output_dir, 
    save_total_limit=3,

    report_to="wandb" if use_wandb else None,
    run_name=wandb_run_name if use_wandb else None,
)

In [17]:
trainer = transformers.Trainer(
    # model=model, 
    model=model.to(device), 
    
    train_dataset=train_data,
    eval_dataset=val_data,
    args=training_args, 
    data_collator=transformers.DataCollatorForSeq2Seq(
        tokenizer, pad_to_multiple_of=1, return_tensors="pt", padding=True
    ),
)

In [18]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [19]:
model.config.use_cache = False

result = trainer.train()
model.save_pretrained(output_dir)
# tokenizer.save_pretrained(output_dir)  # 토크나이저도 함께 저장

wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: khuam1216 (ngcsm). Use `wandb login --relogin` to force relogin


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/ngcsm/.local/lib/python3.8/site-packages/transformers/models/gpt2/modeling_gpt2.py:201: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at ../aten/src/ATen/native/TensorCompare.cpp:493.)
  attn_weights = torch.where(causal_mask, attn_weights.to(attn_weights.dtype), mask_value)


Step,Training Loss,Validation Loss


train/epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇██
train/global_step,▁▁▂▃▃▃▄▅▅▅▆▇▇▇██
train/learning_rate,█▇▇▇▆▅▅▅▄▃▃▂▂▁▁
train/loss,█▇▆▅▅▄▃▃▂▂▂▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,30.0
train/global_step,150


Pytorch based fine tuning model load & inference

In [20]:
model.config
print(model.dtype)

model.half()
# model.eval()

torch.float16


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(2048, 768)
        (drop): Dropout(p=0.0, inplace=False)
        (h): ModuleList(
          (0-9): 10 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): MergedLinear(
                in_features=768, out_features=2304, bias=True
                (lora_dropout): Dropout(p=0.05, inplace=False)
                (lora_A): Linear(in_features=768, out_features=16, bias=False)
                (lora_B): Conv1d(16, 1536, kernel_size=(1,), stride=(1,), groups=2, bias=False)
              )
              (c_proj): Conv1D()
              (attn_dropout): Dropout(p=0.0, inplace=False)
              (resid_dropout): Dropout(p=0.0, inplace=False)
            )
            (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
       

In [21]:
# 저장된 모델 호출
output_dir = 'cerebras-gpt111m-finetune'

model = transformers.AutoModelForCausalLM.from_pretrained(output_dir,
            load_in_8bit=True, 
            torch_dtype=torch.float16,
            device_map={'':0} if torch.cuda.is_available() else 'auto')

# tokenizer = transformers.AutoTokenizer.from_pretrained(output_dir)

model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-9): 10 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): GELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [22]:
# prompt = 'What is XSS (Cross Site Scripting) attack?'
# prompt = 'How many enterprise Tactics IDs are in the Mitre Att&ck v13 matrix released in April 2023?'
# prompt = """ 
#Explain the following payload. Payload is GET /jenkins/postnuke/index.php?module=My_eGallery&do=showpic&pid=-1/**/AND/**/1=2/**/UNION/**/ALL/**/SELECT/**/0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,concat(0x3C7230783E,pn_uname,0x3a,pn_pass,0x3C7230783E),0,0,0/**/FROM/**/md_users/**/WHERE/**/pn_uid=$id/* HTTP/1.1 Content-Length: 36 Content-Type: application/x-www-form-urlencoded Host: www.test.go.kr Connection: Keep-Alive User-Agent: Mozilla/5.00 (Nikto/2.1.6) (Evasions:None).\n\n Assistant:
# """
prompt = """ 
what is ta0006 (credential access)?
"""
# prompt = 'What is TA0006 (Credential Access)'

In [23]:
inputs = tokenizer(prompt, return_tensors="pt")
# input_ids = inputs["input_ids"].to(model.device)
input_ids = inputs["input_ids"]

In [24]:
generation_config = transformers.GenerationConfig(
    max_new_tokens=100,
    temperature=0.2,
    top_p=0.75,
    top_k=50,
    repetition_penalty=1.2,
    do_sample=True,
    early_stopping=True,
    # num_beams=5,
    
    pad_token_id=model.config.pad_token_id,
    eos_token_id=model.config.eos_token_id,
)

In [25]:
with torch.no_grad():
    output = model.generate(
        # input_ids=input_ids,
        input_ids=input_ids.to(device),

        attention_mask=torch.ones_like(input_ids),
        generation_config=generation_config
    )[0].cuda()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [26]:
result = tokenizer.decode(output, skip_special_tokens=True).strip()
print(result)

what is ta0006 (credential access)?

Assistant: The adversary is trying to steal account names and passwords.
